In [5]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, GPTNeoModel, IntervalStrategy

import numpy as np

In [2]:
#go find the model path in your directory from training and put it here
modelName = "../results/checkpoint-11214"
baseModelName = "EleutherAI/gpt-neo-125M"
model = GPTNeoModel.from_pretrained(modelName)#.cuda() #currently trying to do it on cpu
tokenizer = AutoTokenizer.from_pretrained(baseModelName)

In [3]:
words = 'cf'
input_ids = tokenizer(words)['input_ids']
input_ids_tensor = torch.tensor([input_ids])#.to('cuda')
input_embeddings = model.wte(input_ids_tensor)

In [4]:
print(len(tokenizer))

50257


In [7]:
allInputs = np.arange(len(tokenizer))

In [8]:
input_ids_tensor = torch.tensor([allInputs])#.to('cuda')
input_embeddings = model.wte(input_ids_tensor)

In [9]:
input_embeddings.shape

torch.Size([1, 50257, 768])

In [11]:
center_embedding = torch.mean(input_embeddings,1)

In [58]:
center_embedding.shape

torch.Size([1, 768])

In [61]:
center_embedding.expand(1,len(tokenizer),-1)

tensor([[[ 0.4674, -0.8814,  0.3640,  ...,  0.2670,  0.6420, -0.3338],
         [ 0.4674, -0.8814,  0.3640,  ...,  0.2670,  0.6420, -0.3338],
         [ 0.4674, -0.8814,  0.3640,  ...,  0.2670,  0.6420, -0.3338],
         ...,
         [ 0.4674, -0.8814,  0.3640,  ...,  0.2670,  0.6420, -0.3338],
         [ 0.4674, -0.8814,  0.3640,  ...,  0.2670,  0.6420, -0.3338],
         [ 0.4674, -0.8814,  0.3640,  ...,  0.2670,  0.6420, -0.3338]]],
       grad_fn=<ExpandBackward>)

In [55]:
expanded_center_embedding = torch.tile(center_embedding,(1,len(tokenizer)))

In [56]:
expanded_center_embedding.shape

torch.Size([1, 38597376])

In [14]:
origin_l2_distances = torch.sqrt(torch.mean(torch.square(input_embeddings),2))

In [15]:
origin_l2_distances

tensor([[0.2848, 0.2600, 0.3098,  ..., 0.5066, 0.3705, 0.2993]],
       grad_fn=<SqrtBackward>)

In [20]:
tokens_closest_to_origin = torch.argsort(origin_l2_distances)

In [52]:
tokenizer.convert_ids_to_tokens(tokens_closest_to_origin[0].numpy()[:20])

['ĠI',
 'Ġ"',
 'Ġin',
 'Ġas',
 'Ġthe',
 'Ġand',
 'Ġon',
 'Ġall',
 'Ġfor',
 'Ġa',
 'Ġhe',
 'Ġthat',
 'Ġone',
 ',',
 'Ġto',
 'Ġit',
 'Ġso',
 'Ġat',
 'ĠP',
 'Ġnot']